In [1]:
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-837VJMV:4041
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1629995619840)
SparkSession available as 'spark'


import org.apache.spark.sql.functions._


In [28]:
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "LEGACY")

In [15]:
val dfCSV = spark.read.option("header", "true").option("delimiter", ";").csv("C:/Users/BLUETAB/Documents/Bluetab/BluetabUniversity/SesionSpark/bluetabUniversitySpark/clientes.csv")

dfCSV: org.apache.spark.sql.DataFrame = [id: string, first_name: string ... 11 more fields]


In [79]:
val dfReal = dfCSV.withColumn("profession", regexp_replace(col("profession"),"[1]","l"))

dfReal: org.apache.spark.sql.DataFrame = [id: string, first_name: string ... 11 more fields]


In [40]:
val dfCast = dfCSV.select(col("id"), col("email"), col("debt").cast("decimal(15,2)"))

dfCast: org.apache.spark.sql.DataFrame = [id: string, email: string ... 1 more field]


In [43]:
dfCSV.withColumn("debt", col("debt").cast("decimal(15,2)")).printSchema

root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- profession: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- register_date: string (nullable = true)
 |-- partition_field: string (nullable = true)
 |-- debt: decimal(15,2) (nullable = true)
 |-- salary: string (nullable = true)



<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">1. Lectura</span>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">1.1 Leer CSV</span><br>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">1.2 Leer AVRO</span><br>

In [24]:
val df2 = spark.read.format("avro").
load("C:/Users/BLUETAB/Documents/Bluetab/BluetabUniversity/SesionSpark/bluetabUniversitySpark/profession/")

df2: org.apache.spark.sql.DataFrame = [profession: string, Category: string ... 1 more field]


In [91]:
df2.withColumn("fecha", fil6/08/2021")).show

+--------------+--------+---+----------+
|    profession|Category| Sg|     fecha|
+--------------+--------+---+----------+
|        doctor|      AA| Dr|26/08/2021|
|police officer|       B| PO|26/08/2021|
|   firefighter|       C| FF|26/08/2021|
|        worker|       D| Wr|26/08/2021|
|     developer|       B| Dv|26/08/2021|
|         other|       F| Ot|26/08/2021|
+--------------+--------+---+----------+



In [92]:
dfReal.filter(col("age")<70).join(df2, dfReal("profession")===df2("profession")).show

+---+----------+---------+--------------------+----------+---+------------+--------+------------+-------------+---------------+-----------+------+----------+--------+---+
| id|first_name|last_name|               email|profession|age|        city| country|country_code|register_date|partition_field|       debt|salary|profession|Category| Sg|
+---+----------+---------+--------------------+----------+---+------------+--------+------------+-------------+---------------+-----------+------+----------+--------+---+
|100|   Aigneis| Sinegold|Aigneis.Sinegold@...| developer| 65|     Hanover|   Gabon|          AI|   2014-08-22|            003| 9855840.38|     E| developer|       B| Dv|
|102|Marguerite|     Zola|Marguerite.Zola@m...| developer| 21|     Tripoli| Burundi|          JM|   2013-07-02|            001| 9172087.55|     F| developer|       B| Dv|
|104|     Talya| Shirberg|Talya.Shirberg@me...|     other| 38|Philadelphia|Suriname|          SG|   2014-04-19|            001|83293969.58|     H

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">1.3 Leer PARQUET</span><br>

In [29]:
val dfParquet = spark.read.parquet("C:/Users/BLUETAB/Documents/Bluetab/BluetabUniversity/SesionSpark/bluetabUniversitySpark/contracts")

dfParquet: org.apache.spark.sql.DataFrame = [cutoff_date: date, entity_id: string ... 91 more fields]


In [108]:
val dfParquet2 = spark.read.parquet("C:/Users/BLUETAB/Documents/Bluetab/BluetabUniversity/SesionSpark/bluetabUniversitySpark/contractsP/")

dfParquet2: org.apache.spark.sql.DataFrame = [profession: string, Sg: string ... 1 more field]


In [109]:
dfParquet2.printSchema

root
 |-- profession: string (nullable = true)
 |-- Sg: string (nullable = true)
 |-- Category: string (nullable = true)



In [103]:
df2.write.mode("overwrite").partitionBy("Category").parquet("C:/Users/BLUETAB/Documents/Bluetab/BluetabUniversity/SesionSpark/bluetabUniversitySpark/contractsP")

In [94]:
val df3 = dfParquet.select(col("contract_id"), when(col("next_review_date").equalTo("0001-01-01"), null)
        .otherwise(col("next_review_date")).as("fecha"), date_format(col("next_review_date"),"dd-MM-yyyy").alias("fecha_casteada"))

df3: org.apache.spark.sql.DataFrame = [contract_id: string, fecha: date ... 1 more field]


In [95]:
df3.select("fecha_casteada").distinct.show

+--------------+
|fecha_casteada|
+--------------+
|    22-07-2019|
|    13-12-2019|
|    14-09-2019|
|    17-07-2019|
|    25-09-2019|
|    28-07-2019|
|    01-02-2021|
|    12-10-2017|
|    08-08-2019|
|    13-07-2019|
|    20-02-2020|
|    20-08-2019|
|    15-07-2019|
|    16-07-2019|
|    10-09-2019|
|    30-07-2019|
|    26-12-2019|
|    29-08-2019|
|    23-07-2019|
|    19-08-2019|
+--------------+
only showing top 20 rows



<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">2. Conocer los datos</span>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">2.1 Conocer el schema</span><br>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">2.2 Consultar los datos</span><br>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">3. Limpieza de datos</span>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">3.1 Hay valores nulos?</span><br>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">3.2 Hay valores duplicados (distinct, group By, dropDuplicates)?</span><br>

In [51]:
dfParquet.select("contract_id").groupBy("contract_id").agg(count("contract_id").as("cantidad")).show

+------------------+--------+
|       contract_id|cantidad|
+------------------+--------+
|001304014900194188|       2|
|001305894900004886|       2|
|001306204800069286|       2|
|001303034900044184|       2|
|001304014900187687|       2|
|001305004601152086|       2|
|001304014900177282|       2|
|001303054701336082|       4|
|001306204800068981|       2|
|001305005400501382|       2|
|001305004601154678|       2|
|001304015000026778|       2|
|001306914800001481|       2|
|001308335400107881|       2|
|001304014900190681|       2|
|001306204800071076|       2|
|001305005400501572|       2|
|001303054900052177|       2|
|001304010200002274|       1|
|001304015000037072|       1|
+------------------+--------+
only showing top 20 rows



In [55]:
dfParquet.select("contract_id").dropDuplicates.count

res38: Long = 23231


<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">3.3 Formatear la fecha (dd/mm/yyyy)</span><br>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">3.4 Castear datos</span><br>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">4. Cruzar dataframes</span>

<span style="color:gray; font-family: 'Bebas Neue'; font-size: 2em;">4.1 Cambiar nombre de columnas (as and column rename)</span><br>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">5. Filtrar Datos</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">6. Crear nuevos dataFrames</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">7. Particionar</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">8. Agrupar</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">9. Crear nuevas columas</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">10. Borrar columas</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">11. Concatenar columas</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">12. Ordenmiento</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">13. Limitar</span>

<span style="color:blue; font-family: 'Bebas Neue'; font-size: 3em;">14. Expresiones regulares</span>